# Contour extraction for ground truth masks of DAVIS 2016

In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd
import re
from scipy import spatial
from skimage import measure

%matplotlib inline

## Functions

In [2]:
def save_image_with_contour(image, contour, path):
    '''Saves image plotted with contour.'''
    
    fig, ax = plt.subplots()
    ax.scatter(contour[:, 0], contour[:, 1], s=1)
    ax.imshow(image)
    ax.axis('image')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(path, dpi=100, bbox_inches='tight')    
    plt.close(fig)

In [3]:
def close_image(image, kernel_size):
    '''Returns the image that is closed with a kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [4]:
def extract_longest_contour(image, kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    closing = close_image(image, kernel_size)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(gray, 127, 255, 0)

    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    contours, _ = cv2.findContours(image=thresh,
                                   mode=cv2.RETR_TREE,
                                   method=cv2.CHAIN_APPROX_NONE)
    
    # Get longest contour
    longest_contour = max(contours, key=len)
    
    # Remove single-dimensional entry from the shape of the array 
    longest_contour = np.squeeze(longest_contour)
    
    return longest_contour

In [5]:
def shorten_contour(contour, points):
    '''Returns a contour with a fixed number of points for a given contour.'''
    
    # TODO: add warning if short_contour has less than stated points
    try:
        step = len(contour) / (len(contour) - points)
    except ZeroDivisionError as e:
        print ('len(contour) = {}, points = {}'.format(len(contour), points))
        raise ValueError from e
        
    short_contour = np.delete(contour, np.arange(0, contour.size, step), axis=0)
    
    return short_contour

In [6]:
def get_contour_from_annotation(annotation_path, kernel_size, points):
    '''Returns the image, the longest contour, and shortened contour 
       given a annotation and fixed number of points.'''
    
    image = cv2.imread(annotation_path)
    contour = extract_longest_contour(image, kernel_size)
    try:
        short_contour = shorten_contour(contour, points)
    except ValueError as e:
        print(annotation_path)
        raise ValueError from e
        
    return image, contour, short_contour

In [7]:
def create_contours_for_all_annotations(annotations_folders_path, 
                                        contours_folders_path, 
                                        kernel_size,
                                        contour_points):

    annotations_folders_list = os.listdir(annotations_folders_path)
    
    # Iterate through folders
    for i, folder in enumerate(annotations_folders_list):
        
        print('#{}: {}'.format(i, folder))
        
        # Create folder to save contours
        contours_folder = os.path.join(contours_folders_path, folder)
        if not os.path.exists(contours_folder):
            os.makedirs(contours_folder)
        
        # Get list of annotations
        annotations_list = os.listdir(os.path.join(annotations_folders_path, folder))
        if '.ipynb_checkpoints' in annotations_list:
            annotations_list.remove('.ipynb_checkpoints')
            
        # Iterate through annotations
        for j, annotation in enumerate(annotations_list):

            annotation_path = os.path.join(annotations_folders_path, folder, annotation)

            # If first mask, then extract contour_points
            if re.match(r"00000.png", annotation):
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points)
                except ValueError as e:
                    continue
            # If not, then contour_points*2
            else:
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points * 2)
                except ValueError as e:
                    continue

            # Save contour
            save_image_with_contour(image, short_contour, os.path.join(contours_folder, annotation))
            np.save(os.path.join(contours_folder, annotation), short_contour)

## Create Contours

In [8]:
ANNOTATIONS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
CONTOURS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/480p/"

ANNOTATIONS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/1080p/"
CONTOURS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/1080p/"

KERNEL_SIZE = 10
CONTOUR_POINTS = 128

create_contours_for_all_annotations(ANNOTATIONS_480P_FOLDERS_PATH,
                                    CONTOURS_480P_FOLDERS_PATH,
                                    KERNEL_SIZE, 
                                    CONTOUR_POINTS)

#0: swing


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  if sys.path[0] == '':


#1: drift-chicane
#2: lucia
#3: soapbox
#4: breakdance
#5: drift-turn
#6: mallard-fly
#7: motorbike
#8: scooter-gray
#9: scooter-black
#10: breakdance-flare
#11: bus
#12: elephant
#13: bmx-trees
#14: rollerblade
#15: dance-twirl
#16: dance-jump
#17: horsejump-high
#18: mallard-water
#19: car-turn
#20: kite-walk
#21: dog-agility
#22: car-shadow
#23: paragliding-launch
#24: stroller
#25: bear
#26: hockey
#27: dog
#28: boat
#29: car-roundabout
#30: soccerball
contour: [[115 267]
 [114 268]
 [113 268]
 [112 268]
 [111 268]
 [110 268]
 [109 268]
 [108 269]
 [107 269]
 [106 269]
 [105 270]
 [104 270]
 [103 271]
 [102 272]
 [101 272]
 [100 273]
 [ 99 274]
 [ 98 274]
 [ 97 275]
 [ 96 276]
 [ 95 277]
 [ 94 278]
 [ 94 279]
 [ 93 280]
 [ 92 281]
 [ 91 282]
 [ 90 283]
 [ 89 284]
 [ 89 285]
 [ 88 286]
 [ 87 287]
 [ 87 288]
 [ 86 289]
 [ 85 290]
 [ 85 291]
 [ 85 292]
 [ 84 293]
 [ 84 294]
 [ 84 295]
 [ 83 296]
 [ 83 297]
 [ 83 298]
 [ 83 299]
 [ 83 300]
 [ 83 301]
 [ 83 302]
 [ 83 303]
 [ 82 304]
 [